# A2C 강화학습

Pre-trained ChessCNN 모델을 A2C (Advantage Actor-Critic) 알고리즘과 Self-play로 강화학습합니다.

## 알고리즘 개요
- **Advantage**: A(s,a) = R - V(s)
- **Policy Loss**: -log π(a|s) × A(s,a)
- **Value Loss**: (V(s) - R)²
- **Self-play**: 현재 네트워크 vs 현재 네트워크

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import chess
from pathlib import Path
from tqdm import tqdm
from collections import deque
import random
import time
import copy

from preprocessing import (
    board_to_tensor,
    legal_move_mask,
    action_index_to_move,
    move_to_action_index,
    ACTION_SPACE_SIZE
)
from chess_model import ChessCNN, load_model
from train_utils import compute_masked_entropy, get_masked_log_probs

# 시드 고정
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# 디바이스 설정
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    device = torch.device("cuda")
    print(f"✅ CUDA 사용 가능!")
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    device = torch.device("cpu")
    print(f"⚠️  CUDA 사용 불가 - CPU 사용")

print(f"\n사용 장치: {device}")

✅ CUDA 사용 가능!
   GPU: NVIDIA GeForce RTX 5060 Ti
   GPU 메모리: 15.93 GB

사용 장치: cuda


## 모델 정의 및 Pre-trained 가중치 로드

In [2]:
# ChessCNN은 chess_model.py에서 import됨
# Pre-trained 가중치 로드
PRETRAINED_PATH = Path("models/best_chess_cnn.pth")

if PRETRAINED_PATH.exists():
    model, checkpoint = load_model(PRETRAINED_PATH, device=device)
    print(f"✅ Pre-trained 모델 로드 완료!")
    print(f"   원본 Epoch: {checkpoint.get('epoch', 'N/A')}")
    print(f"   원본 Val Loss: {checkpoint.get('val_loss', 'N/A'):.4f}")
    print(f"   원본 Val Accuracy: {checkpoint.get('val_accuracy', 'N/A')*100:.2f}%")
else:
    # Pre-trained 모델이 없으면 랜덤 초기화
    model = ChessCNN(num_channels=256).to(device)
    print(f"⚠️  Pre-trained 모델을 찾을 수 없습니다: {PRETRAINED_PATH}")
    print(f"   랜덤 초기화로 시작합니다.")

# 강화학습을 위해 학습 모드로 전환
model.train()

print(f"\n모델 파라미터 수: {sum(p.numel() for p in model.parameters()):,}")

✅ Pre-trained 모델 로드 완료!
   원본 Epoch: 5
   원본 Val Loss: 2.6870
   원본 Val Accuracy: 44.69%

모델 파라미터 수: 9,314,433


## Self-play 함수 구현

In [3]:
def select_action(policy_logits, mask, temperature=1.0):
    """
    정책에서 행동을 샘플링합니다.
    
    Args:
        policy_logits: (4096,) 정책 로짓
        mask: (4096,) 합법 수 마스크
        temperature: 탐색 온도 (높을수록 랜덤, 낮을수록 greedy)
    
    Returns:
        action: 선택된 액션 인덱스
        log_prob: 선택된 액션의 log 확률
        probs: 전체 확률 분포
    """
    # 불법 수 마스킹
    masked_logits = policy_logits.clone()
    masked_logits[~mask.bool()] = float('-inf')
    
    # Temperature 적용
    if temperature != 1.0:
        masked_logits = masked_logits / temperature
    
    # Softmax로 확률 계산
    probs = F.softmax(masked_logits, dim=-1)
    
    # 확률적 샘플링
    action = torch.multinomial(probs, 1).item()
    log_prob = torch.log(probs[action] + 1e-10)
    
    return action, log_prob, probs


def get_game_result(board: chess.Board) -> float:
    """
    게임 결과를 반환합니다.
    
    Returns:
        +1.0: 백 승
        -1.0: 흑 승
         0.0: 무승부
    """
    result = board.result()
    if result == "1-0":
        return 1.0
    elif result == "0-1":
        return -1.0
    else:
        return 0.0


def find_promotion_move(board: chess.Board, action_idx: int) -> chess.Move:
    """
    프로모션이 필요한 경우 Queen 프로모션으로 변환합니다.
    """
    base_move = action_index_to_move(action_idx)
    
    # 합법 수 중에서 같은 from/to를 가진 수 찾기
    for move in board.legal_moves:
        if move.from_square == base_move.from_square and move.to_square == base_move.to_square:
            # 프로모션인 경우 Queen 승격만 허용
            if move.promotion is not None:
                return chess.Move(move.from_square, move.to_square, promotion=chess.QUEEN)
            return move
    
    # 찾지 못하면 기본 수 반환 (에러 상황)
    return base_move


@torch.no_grad()
def play_game(model, device, temperature=1.0, max_moves=200):
    """
    Self-play로 한 게임을 진행합니다.
    
    Args:
        model: ChessCNN 모델
        device: 연산 장치
        temperature: 탐색 온도
        max_moves: 최대 수 제한 (무한 루프 방지)
    
    Returns:
        trajectory: [(state_tensor, action, log_prob, value), ...]
        result: 게임 결과 (+1, 0, -1)
    """
    model.eval()
    board = chess.Board()
    trajectory = []
    
    move_count = 0
    while not board.is_game_over() and move_count < max_moves:
        # 상태를 텐서로 변환
        state = board_to_tensor(board)
        mask = legal_move_mask(board)
        
        state_tensor = torch.from_numpy(state).unsqueeze(0).to(device)
        mask_tensor = torch.from_numpy(mask).unsqueeze(0).to(device)
        
        # 모델 추론
        policy_logits, value = model(state_tensor, mask_tensor)
        policy_logits = policy_logits.squeeze(0)
        value = value.squeeze()
        
        # 행동 선택
        action, log_prob, _ = select_action(policy_logits, mask_tensor.squeeze(0), temperature)
        
        # trajectory에 저장 (나중에 학습에 사용)
        trajectory.append({
            'state': state_tensor.squeeze(0),  # (18, 8, 8)
            'mask': mask_tensor.squeeze(0),    # (4096,)
            'action': action,
            'log_prob': log_prob,
            'value': value,
            'turn': board.turn  # WHITE=True, BLACK=False
        })
        
        # 수 실행
        move = find_promotion_move(board, action)
        board.push(move)
        move_count += 1
    
    # 게임 결과 (무승부 처리)
    if move_count >= max_moves:
        result = 0.0  # 최대 수 초과 시 무승부 처리
    else:
        result = get_game_result(board)
    
    return trajectory, result, board


# =============================================================================
# 배치 GPU 추론을 위한 Vectorized Environment
# =============================================================================

class VectorizedChessEnv:
    """
    여러 체스 게임을 동시에 관리하는 벡터화된 환경
    
    모든 게임의 상태를 배치로 모아서 GPU에서 한 번에 추론합니다.
    """
    def __init__(self, num_envs, max_moves=200):
        self.num_envs = num_envs
        self.max_moves = max_moves
        self.boards = [chess.Board() for _ in range(num_envs)]
        self.trajectories = [[] for _ in range(num_envs)]
        self.dones = [False] * num_envs
        self.move_counts = [0] * num_envs
    
    def get_states_batch(self, device):
        """
        모든 활성 게임의 보드 상태를 배치 텐서로 반환합니다.
        
        Returns:
            states: (num_active, 18, 8, 8) 텐서
            masks: (num_active, 4096) 텐서
            active_indices: 활성 게임 인덱스 리스트
        """
        active_indices = [i for i in range(self.num_envs) if not self.dones[i]]
        
        if not active_indices:
            return None, None, []
        
        states = []
        masks = []
        
        for idx in active_indices:
            state = board_to_tensor(self.boards[idx])
            mask = legal_move_mask(self.boards[idx])
            states.append(state)
            masks.append(mask)
        
        # 배치 텐서로 변환
        states_tensor = torch.from_numpy(np.stack(states)).to(device)  # (num_active, 18, 8, 8)
        masks_tensor = torch.from_numpy(np.stack(masks)).to(device)    # (num_active, 4096)
        
        return states_tensor, masks_tensor, active_indices
    
    def step_batch(self, actions, policy_logits_batch, values_batch, active_indices, temperature):
        """
        배치로 받은 액션을 각 게임에 적용합니다.
        
        Args:
            actions: (num_active,) 액션 인덱스 리스트
            policy_logits_batch: (num_active, 4096) 정책 로짓 배치
            values_batch: (num_active, 1) 가치 배치
            active_indices: 활성 게임 인덱스 리스트
            temperature: 탐색 온도
        """
        for batch_idx, env_idx in enumerate(active_indices):
            if self.dones[env_idx]:
                continue
            
            board = self.boards[env_idx]
            
            # 상태와 마스크 (trajectory 저장용)
            state = board_to_tensor(board)
            mask = legal_move_mask(board)
            state_tensor = torch.from_numpy(state).to(policy_logits_batch.device)
            mask_tensor = torch.from_numpy(mask).to(policy_logits_batch.device)
            
            # 액션 선택 (배치에서 해당 인덱스 추출)
            policy_logits = policy_logits_batch[batch_idx]  # (4096,)
            value = values_batch[batch_idx].item() if values_batch.dim() > 0 else values_batch[batch_idx]
            action = actions[batch_idx]
            
            # Log 확률 계산 (trajectory 저장용)
            masked_logits = policy_logits.clone()
            masked_logits[~mask_tensor.bool()] = float('-inf')
            if temperature != 1.0:
                masked_logits = masked_logits / temperature
            probs = F.softmax(masked_logits, dim=-1)
            log_prob = torch.log(probs[action] + 1e-10)
            
            # Trajectory 저장 (GPU에 그대로 저장)
            self.trajectories[env_idx].append({
                'state': state_tensor,
                'mask': mask_tensor,
                'action': action,
                'log_prob': log_prob,
                'value': value,
                'turn': board.turn
            })
            
            # 수 실행
            move = find_promotion_move(board, action)
            board.push(move)
            self.move_counts[env_idx] += 1
            
            # 게임 종료 체크
            if board.is_game_over() or self.move_counts[env_idx] >= self.max_moves:
                self.dones[env_idx] = True
    
    def get_trajectories(self):
        """모든 게임의 trajectory 반환"""
        return self.trajectories
    
    def get_results(self):
        """모든 게임의 결과 반환"""
        results = []
        for i in range(self.num_envs):
            if self.move_counts[i] >= self.max_moves:
                results.append(0.0)  # 최대 수 초과 시 무승부
            else:
                results.append(get_game_result(self.boards[i]))
        return results
    
    def all_done(self):
        """모든 게임이 종료되었는지 확인"""
        return all(self.dones)


def select_actions_batch(policy_logits_batch, masks_batch, temperature=1.0):
    """
    배치로 정책에서 행동을 샘플링합니다.
    
    Args:
        policy_logits_batch: (batch_size, 4096) 정책 로짓 배치
        masks_batch: (batch_size, 4096) 합법 수 마스크 배치
        temperature: 탐색 온도
    
    Returns:
        actions: (batch_size,) 선택된 액션 인덱스 리스트
    """
    batch_size = policy_logits_batch.size(0)
    actions = []
    
    for i in range(batch_size):
        policy_logits = policy_logits_batch[i]  # (4096,)
        mask = masks_batch[i]  # (4096,)
        
        # 불법 수 마스킹
        masked_logits = policy_logits.clone()
        masked_logits[~mask.bool()] = float('-inf')
        
        # Temperature 적용
        if temperature != 1.0:
            masked_logits = masked_logits / temperature
        
        # Softmax로 확률 계산
        probs = F.softmax(masked_logits, dim=-1)
        
        # 확률적 샘플링
        action = torch.multinomial(probs, 1).item()
        actions.append(action)
    
    return actions


@torch.no_grad()
def play_games_vectorized(model, device, num_games, temperature=1.0, max_moves=200):
    """
    배치 GPU 추론으로 여러 게임을 동시에 진행합니다.
    
    Args:
        model: ChessCNN 모델
        device: 연산 장치
        num_games: 동시에 진행할 게임 수
        temperature: 탐색 온도
        max_moves: 최대 수 제한
    
    Returns:
        all_trajectories: 모든 게임의 trajectory 리스트
        results: 각 게임의 결과
        stats: 통계 정보
    """
    model.eval()
    env = VectorizedChessEnv(num_games, max_moves)
    
    # 모든 게임이 종료될 때까지 반복
    while not env.all_done():
        # 1. 모든 활성 게임의 상태를 배치로 수집
        states_batch, masks_batch, active_indices = env.get_states_batch(device)
        
        if states_batch is None:
            break
        
        # 2. GPU에서 배치 추론
        policy_logits_batch, values_batch = model(states_batch, masks_batch)
        # policy_logits_batch: (num_active, 4096)
        # values_batch: (num_active, 1)
        
        # 3. 각 게임에 대해 액션 선택
        actions = select_actions_batch(policy_logits_batch, masks_batch, temperature)
        
        # 4. 액션 적용
        env.step_batch(actions, policy_logits_batch, values_batch, active_indices, temperature)
    
    # 결과 수집
    all_trajectories = env.get_trajectories()
    results = env.get_results()
    
    # 통계 계산
    total_moves = sum(len(traj) for traj in all_trajectories)
    white_wins = sum(1 for r in results if r > 0)
    black_wins = sum(1 for r in results if r < 0)
    draws = sum(1 for r in results if r == 0)
    
    stats = {
        'num_games': num_games,
        'avg_moves': total_moves / num_games if num_games > 0 else 0,
        'white_wins': white_wins,
        'black_wins': black_wins,
        'draws': draws
    }
    
    return all_trajectories, results, stats


def play_games_sequential(model, device, num_games, temperature=1.0, max_moves=200):
    """
    순차적으로 여러 게임을 진행합니다 (기존 로직, 롤백용).
    
    Returns:
        all_trajectories: 모든 게임의 trajectory 리스트
        results: 각 게임의 결과
        stats: 통계 정보
    """
    all_trajectories = []
    results = []
    total_moves = 0
    white_wins = 0
    black_wins = 0
    draws = 0
    
    for _ in range(num_games):
        trajectory, result, board = play_game(model, device, temperature, max_moves)
        all_trajectories.append(trajectory)
        results.append(result)
        total_moves += len(trajectory)
        
        if result > 0:
            white_wins += 1
        elif result < 0:
            black_wins += 1
        else:
            draws += 1
    
    stats = {
        'num_games': num_games,
        'avg_moves': total_moves / num_games,
        'white_wins': white_wins,
        'black_wins': black_wins,
        'draws': draws
    }
    
    return all_trajectories, results, stats


def play_games_batch(model, device, num_games, temperature=1.0, num_envs=1, max_moves=200):
    """
    여러 게임을 Self-play로 진행합니다.
    
    Args:
        model: ChessCNN 모델
        device: 연산 장치
        num_games: 총 게임 수
        temperature: 탐색 온도
        num_envs: 동시에 진행할 게임 수 (배치 크기)
                  num_envs=1: 순차 실행 (기존 동작, 롤백용)
                  num_envs>1: 배치 GPU 추론
        max_moves: 최대 수 제한
    
    Returns:
        all_trajectories: 모든 게임의 trajectory 리스트
        results: 각 게임의 결과
        stats: 통계 정보
    """
    if num_envs == 1:
        # 순차 실행 (기존 동작, 롤백용)
        return play_games_sequential(model, device, num_games, temperature, max_moves)
    else:
        # 배치 GPU 추론
        # num_games를 num_envs 단위로 나누어 처리
        all_trajectories = []
        all_results = []
        total_moves = 0
        white_wins = 0
        black_wins = 0
        draws = 0
        
        num_batches = (num_games + num_envs - 1) // num_envs  # 올림
        
        for batch_idx in range(num_batches):
            batch_size = min(num_envs, num_games - batch_idx * num_envs)
            if batch_size <= 0:
                break
            
            trajectories, results, stats = play_games_vectorized(
                model, device, batch_size, temperature, max_moves
            )
            
            all_trajectories.extend(trajectories)
            all_results.extend(results)
            total_moves += stats['avg_moves'] * batch_size
            white_wins += stats['white_wins']
            black_wins += stats['black_wins']
            draws += stats['draws']
        
        final_stats = {
            'num_games': num_games,
            'avg_moves': total_moves / num_games if num_games > 0 else 0,
            'white_wins': white_wins,
            'black_wins': black_wins,
            'draws': draws
        }
        
        return all_trajectories, all_results, final_stats


# =============================================================================
# 모델 평가 함수 (승률 기반 저장용)
# =============================================================================

@torch.no_grad()
def play_game_two_models(current_model, opponent_model, device, temperature=0.5, max_moves=200):
    """
    두 모델이 대결하는 게임을 진행합니다.
    
    Args:
        current_model: 현재 모델 (평가 대상)
        opponent_model: 상대 모델
        device: 연산 장치
        temperature: 탐색 온도
        max_moves: 최대 수 제한
    
    Returns:
        result: 게임 결과 (+1=현재 모델 승, -1=상대 승, 0=무승부)
    """
    current_model.eval()
    opponent_model.eval()
    board = chess.Board()
    
    move_count = 0
    while not board.is_game_over() and move_count < max_moves:
        # 상태를 텐서로 변환
        state = board_to_tensor(board)
        mask = legal_move_mask(board)
        
        state_tensor = torch.from_numpy(state).unsqueeze(0).to(device)
        mask_tensor = torch.from_numpy(mask).unsqueeze(0).to(device)
        
        # 현재 차례의 모델 선택
        if board.turn == chess.WHITE:
            # 백 차례: 현재 모델
            policy_logits, _ = current_model(state_tensor, mask_tensor)
        else:
            # 흑 차례: 상대 모델
            policy_logits, _ = opponent_model(state_tensor, mask_tensor)
        
        policy_logits = policy_logits.squeeze(0)
        
        # 행동 선택
        action, _, _ = select_action(policy_logits, mask_tensor.squeeze(0), temperature)
        
        # 수 실행
        move = find_promotion_move(board, action)
        board.push(move)
        move_count += 1
    
    # 게임 결과 (현재 모델 관점)
    if move_count >= max_moves:
        result = 0.0  # 최대 수 초과 시 무승부
    else:
        game_result = get_game_result(board)
        # 현재 모델이 백이었으면 game_result 그대로, 흑이었으면 -game_result
        # (play_game_two_models는 항상 현재 모델이 백으로 시작)
        result = game_result
    
    return result


@torch.no_grad()
def evaluate_vs_opponent(current_model, opponent_model, device, num_games=10, temperature=0.5):
    """
    현재 모델 vs 상대 모델 대결
    
    Args:
        current_model: 현재 모델 (평가 대상)
        opponent_model: 상대 모델
        device: 연산 장치
        num_games: 대결 게임 수
        temperature: 탐색 온도
    
    Returns:
        win_rate: 현재 모델의 승률 (0.0 ~ 1.0, 무승부 = 0.5점)
    """
    current_model.eval()
    opponent_model.eval()
    
    total_score = 0.0
    
    for game_idx in range(num_games):
        # 현재 모델이 백/흑 번갈아 플레이
        if game_idx % 2 == 0:
            # 현재 모델이 백
            result = play_game_two_models(current_model, opponent_model, device, temperature)
        else:
            # 현재 모델이 흑 (상대가 백)
            result = play_game_two_models(opponent_model, current_model, device, temperature)
            result = -result  # 결과 반전 (현재 모델 관점)
        
        # 점수 계산: 승=1.0, 무=0.5, 패=0.0
        if result > 0:
            score = 1.0
        elif result < 0:
            score = 0.0
        else:
            score = 0.5
        
        total_score += score
    
    win_rate = total_score / num_games
    return win_rate


# 테스트: 1 게임 플레이
print("테스트 게임 진행 중...")
trajectory, result, board = play_game(model, device, temperature=1.0)
print(f"게임 결과: {result} (1=백승, -1=흑승, 0=무승부)")
print(f"총 수: {len(trajectory)}")
print(f"최종 보드:\n{board}")

게임 결과: 0.0 (1=백승, -1=흑승, 0=무승부)
총 수: 80
최종 보드:
. . . . . . . k
p b . . . p . p
. . . p . . . .
. . p . . . . .
. . . . . n . .
. . . . . . . .
. . . . r . r .
. . . . . K . .


## A2C 학습 함수

In [4]:
def compute_a2c_loss(model, trajectories, results, device, 
                      value_loss_weight=0.5, entropy_bonus=0.01):
    """
    A2C 손실을 계산합니다.
    
    Args:
        model: ChessCNN 모델
        trajectories: 게임 trajectory 리스트
        results: 각 게임의 결과 리스트
        value_loss_weight: Value Loss 가중치
        entropy_bonus: 엔트로피 보너스 (탐색 장려)
    
    Returns:
        total_loss: 전체 손실
        policy_loss: 정책 손실
        value_loss: 가치 손실
        entropy: 평균 엔트로피
    """
    model.train()
    
    total_policy_loss = 0.0
    total_value_loss = 0.0
    total_entropy = 0.0
    total_steps = 0
    
    for trajectory, result in zip(trajectories, results):
        for i, step in enumerate(trajectory):
            state = step['state'].unsqueeze(0)  # (1, 18, 8, 8)
            mask = step['mask'].unsqueeze(0)    # (1, 4096)
            action = step['action']
            turn = step['turn']  # WHITE=True, BLACK=False
            
            # 현재 플레이어 관점의 결과
            # 백 차례면 result 그대로, 흑 차례면 -result
            player_result = result if turn else -result
            
            # Forward pass
            policy_logits, value = model(state, mask)
            policy_logits = policy_logits.squeeze(0)  # (4096,)
            value = value.squeeze()  # scalar
            
            # 마스킹된 log 확률 계산 (공통 함수 사용)
            mask_squeezed = mask.squeeze(0)  # (4096,)
            log_probs = get_masked_log_probs(policy_logits, mask_squeezed)
            
            # Advantage = Return - Value (baseline)
            advantage = player_result - value.detach()
            
            # Policy Loss: -log π(a|s) × A(s,a)
            policy_loss = -log_probs[action] * advantage
            
            # Value Loss: MSE(V(s), R)
            value_loss = (value - player_result) ** 2
            
            # Entropy: 합법 수에 대해서만 정규화된 엔트로피 계산 (공통 함수 사용)
            entropy = compute_masked_entropy(policy_logits, mask_squeezed, device)
            
            total_policy_loss += policy_loss
            total_value_loss += value_loss
            total_entropy += entropy
            total_steps += 1
    
    # 평균 계산
    avg_policy_loss = total_policy_loss / total_steps
    avg_value_loss = total_value_loss / total_steps
    avg_entropy = total_entropy / total_steps
    
    # Total Loss = Policy Loss + c1 * Value Loss - c2 * Entropy
    total_loss = avg_policy_loss + value_loss_weight * avg_value_loss - entropy_bonus * avg_entropy
    
    return total_loss, avg_policy_loss, avg_value_loss, avg_entropy


def train_step(model, optimizer, trajectories, results, device,
               value_loss_weight=0.5, entropy_bonus=0.01, max_grad_norm=1.0):
    """
    한 번의 학습 스텝을 수행합니다.
    """
    optimizer.zero_grad()
    
    total_loss, policy_loss, value_loss, entropy = compute_a2c_loss(
        model, trajectories, results, device, value_loss_weight, entropy_bonus
    )
    
    total_loss.backward()
    
    # Gradient Clipping
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_grad_norm)
    
    optimizer.step()
    
    return {
        'total_loss': total_loss.item(),
        'policy_loss': policy_loss.item(),
        'value_loss': value_loss.item(),
        'entropy': entropy.item()
    }


print("A2C 학습 함수 정의 완료!")

A2C 학습 함수 정의 완료!


## 학습 설정

In [5]:
# =============================================================================
# 학습 하이퍼파라미터
# =============================================================================
LEARNING_RATE = 1e-4           # Pre-trained 모델 fine-tuning용 낮은 학습률
NUM_ITERATIONS = 10000          # 학습 반복 횟수
GAMES_PER_ITERATION = 12       # 반복당 Self-play 게임 수
NUM_ENVS = 12                   # 동시에 진행할 게임 수 (배치 크기)
                                # num_envs=1: 순차 실행 (기존 동작, 롤백용)
                                # num_envs>1: 배치 GPU 추론 (32 권장)
EVAL_GAMES = 10                # 평가용 게임 수
EVAL_INTERVAL = 10             # 평가 주기 (iteration)
WIN_RATE_THRESHOLD = 0.5       # 승률 기준 (50% 이상이면 저장)
TEMPERATURE_START = 1.0        # 시작 탐색 온도
TEMPERATURE_END = 0.5          # 종료 탐색 온도
VALUE_LOSS_WEIGHT = 0.5        # Value Loss 가중치
ENTROPY_BONUS = 0.01           # 엔트로피 보너스
MAX_GRAD_NORM = 1.0            # Gradient Clipping

# 모델 저장 경로
MODEL_DIR = Path("models")
MODEL_DIR.mkdir(exist_ok=True)
RL_MODEL_PATH = MODEL_DIR / "chess_cnn_rl_a2c.pth"
BEST_RL_MODEL_PATH = MODEL_DIR / "best_chess_cnn_rl_a2c.pth"

# TensorBoard 로그
LOG_DIR = MODEL_DIR / "tensorboard_rl"
writer = SummaryWriter(log_dir=LOG_DIR)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Learning Rate Scheduler (optional)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

print("=" * 60)
print("A2C 강화학습 설정")
print("=" * 60)
print(f"  Learning Rate: {LEARNING_RATE}")
print(f"  Iterations: {NUM_ITERATIONS}")
print(f"  Games per Iteration: {GAMES_PER_ITERATION}")
print(f"  Num Envs (Batch Size): {NUM_ENVS} {'(순차 실행)' if NUM_ENVS == 1 else '(배치 GPU 추론)'}")
print(f"  Evaluation: {EVAL_GAMES} games every {EVAL_INTERVAL} iterations (승률 {WIN_RATE_THRESHOLD*100:.0f}%+)")
print(f"  Temperature: {TEMPERATURE_START} → {TEMPERATURE_END}")
print(f"  Value Loss Weight: {VALUE_LOSS_WEIGHT}")
print(f"  Entropy Bonus: {ENTROPY_BONUS}")
print(f"  Gradient Clipping: {MAX_GRAD_NORM}")
print(f"\nTensorBoard: tensorboard --logdir={LOG_DIR}")

A2C 강화학습 설정
  Learning Rate: 0.0001
  Iterations: 10000
  Games per Iteration: 12
  Num Envs (Batch Size): 12 (배치 GPU 추론)
  Evaluation: 10 games every 10 iterations (승률 50%+)
  Temperature: 1.0 → 0.5
  Value Loss Weight: 0.5
  Entropy Bonus: 0.01
  Gradient Clipping: 1.0

TensorBoard: tensorboard --logdir=models/tensorboard_rl


## 학습 루프

In [ ]:
# 학습 히스토리
history = {
    'total_loss': [],
    'policy_loss': [],
    'value_loss': [],
    'entropy': [],
    'avg_moves': [],
    'white_wins': [],
    'black_wins': [],
    'draws': []
}

# 이전 최고 모델 저장용 (초기값은 현재 모델)
best_model_state = copy.deepcopy(model.state_dict())

print("=" * 60)
print("A2C 강화학습 시작")
print("=" * 60)

for iteration in range(1, NUM_ITERATIONS + 1):
    start_time = time.time()
    
    # Temperature 스케줄링 (선형 감소)
    progress = (iteration - 1) / max(NUM_ITERATIONS - 1, 1)
    temperature = TEMPERATURE_START + (TEMPERATURE_END - TEMPERATURE_START) * progress
    
    # 1. Self-play로 게임 생성
    trajectories, results, stats = play_games_batch(
        model, device, GAMES_PER_ITERATION, temperature, num_envs=NUM_ENVS
    )
    
    # 2. A2C 학습
    metrics = train_step(
        model, optimizer, trajectories, results, device,
        VALUE_LOSS_WEIGHT, ENTROPY_BONUS, MAX_GRAD_NORM
    )
    
    # 3. Learning Rate 스케줄링
    scheduler.step()
    
    # 4. 히스토리 저장
    history['total_loss'].append(metrics['total_loss'])
    history['policy_loss'].append(metrics['policy_loss'])
    history['value_loss'].append(metrics['value_loss'])
    history['entropy'].append(metrics['entropy'])
    history['avg_moves'].append(stats['avg_moves'])
    history['white_wins'].append(stats['white_wins'])
    history['black_wins'].append(stats['black_wins'])
    history['draws'].append(stats['draws'])
    
    # 5. TensorBoard 로깅
    writer.add_scalar('Loss/Total', metrics['total_loss'], iteration)
    writer.add_scalar('Loss/Policy', metrics['policy_loss'], iteration)
    writer.add_scalar('Loss/Value', metrics['value_loss'], iteration)
    writer.add_scalar('Entropy', metrics['entropy'], iteration)
    writer.add_scalar('Games/Avg_Moves', stats['avg_moves'], iteration)
    writer.add_scalar('Games/White_Wins', stats['white_wins'], iteration)
    writer.add_scalar('Games/Black_Wins', stats['black_wins'], iteration)
    writer.add_scalar('Games/Draws', stats['draws'], iteration)
    writer.add_scalar('Temperature', temperature, iteration)
    writer.add_scalar('Learning_Rate', optimizer.param_groups[0]['lr'], iteration)
    
    elapsed = time.time() - start_time
    
    # 6. 진행 상황 출력
    if iteration % 5 == 0 or iteration == 1:
        print(f"\nIteration {iteration}/{NUM_ITERATIONS} ({elapsed:.1f}s)")
        print(f"  Loss: {metrics['total_loss']:.4f} (P: {metrics['policy_loss']:.4f}, V: {metrics['value_loss']:.4f})")
        print(f"  Entropy: {metrics['entropy']:.4f}, Temperature: {temperature:.2f}")
        print(f"  Games: W{stats['white_wins']}/B{stats['black_wins']}/D{stats['draws']}, Avg Moves: {stats['avg_moves']:.1f}")
    
    # 7. 승률 기반 최고 모델 저장
    if iteration % EVAL_INTERVAL == 0:
        # 이전 최고 모델 로드
        opponent = ChessCNN(num_channels=256).to(device)
        opponent.load_state_dict(best_model_state)
        opponent.eval()
        
        # 대결
        win_rate = evaluate_vs_opponent(model, opponent, device, EVAL_GAMES, temperature=0.5)
        
        if win_rate >= WIN_RATE_THRESHOLD:
            best_model_state = copy.deepcopy(model.state_dict())
            torch.save({
                'iteration': iteration,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': metrics['total_loss'],
                'win_rate': win_rate,
                'temperature': temperature,
            }, BEST_RL_MODEL_PATH)
            if iteration % 5 == 0 or iteration == 1:
                print(f"  ✅ 새 최고 모델! 승률: {win_rate*100:.1f}%")
        else:
            if iteration % 5 == 0 or iteration == 1:
                print(f"  ⚠️  승률 부족: {win_rate*100:.1f}% < {WIN_RATE_THRESHOLD*100:.0f}%")
    
    # 8. 주기적 체크포인트 저장
    if iteration % 10 == 0:
        torch.save({
            'iteration': iteration,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss': metrics['total_loss'],
            'history': history,
        }, RL_MODEL_PATH)

# 최종 모델 저장
torch.save({
    'iteration': NUM_ITERATIONS,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'loss': history['total_loss'][-1],
    'history': history,
}, RL_MODEL_PATH)

writer.close()

print("\n" + "=" * 60)
print("학습 완료!")
print("=" * 60)
print(f"최종 Loss: {history['total_loss'][-1]:.4f}")
print(f"모델 저장: {RL_MODEL_PATH}")
print(f"최고 모델: {BEST_RL_MODEL_PATH}")


Iteration 1/10000 (4.0s)
  Loss: -1.1078 (P: -1.3930, V: 0.5730)
  Entropy: 0.1258, Temperature: 1.00
  Games: W3/B6/D3, Avg Moves: 106.8


## 학습 곡선 시각화

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 1. Total Loss
axes[0, 0].plot(history['total_loss'], label='Total Loss', color='blue')
axes[0, 0].set_title('Total Loss')
axes[0, 0].set_xlabel('Iteration')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True)

# 2. Policy & Value Loss
axes[0, 1].plot(history['policy_loss'], label='Policy Loss', color='green')
axes[0, 1].plot(history['value_loss'], label='Value Loss', color='red')
axes[0, 1].set_title('Policy & Value Loss')
axes[0, 1].set_xlabel('Iteration')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True)

# 3. Entropy
axes[0, 2].plot(history['entropy'], label='Entropy', color='purple')
axes[0, 2].set_title('Policy Entropy')
axes[0, 2].set_xlabel('Iteration')
axes[0, 2].set_ylabel('Entropy')
axes[0, 2].legend()
axes[0, 2].grid(True)

# 4. Average Moves per Game
axes[1, 0].plot(history['avg_moves'], label='Avg Moves', color='orange')
axes[1, 0].set_title('Average Moves per Game')
axes[1, 0].set_xlabel('Iteration')
axes[1, 0].set_ylabel('Moves')
axes[1, 0].legend()
axes[1, 0].grid(True)

# 5. Win/Loss/Draw
iterations = range(1, len(history['white_wins']) + 1)
axes[1, 1].plot(iterations, history['white_wins'], label='White Wins', color='white', 
                marker='o', markerfacecolor='gray', markeredgecolor='black')
axes[1, 1].plot(iterations, history['black_wins'], label='Black Wins', color='black', marker='s')
axes[1, 1].plot(iterations, history['draws'], label='Draws', color='gray', marker='^')
axes[1, 1].set_title('Game Results')
axes[1, 1].set_xlabel('Iteration')
axes[1, 1].set_ylabel('Count')
axes[1, 1].legend()
axes[1, 1].grid(True)
axes[1, 1].set_facecolor('#f0f0f0')

# 6. Win Rate (Stacked)
total_games = [w + b + d for w, b, d in zip(history['white_wins'], history['black_wins'], history['draws'])]
white_rate = [w / t * 100 for w, t in zip(history['white_wins'], total_games)]
black_rate = [b / t * 100 for b, t in zip(history['black_wins'], total_games)]
draw_rate = [d / t * 100 for d, t in zip(history['draws'], total_games)]

axes[1, 2].stackplot(iterations, white_rate, black_rate, draw_rate, 
                     labels=['White', 'Black', 'Draw'],
                     colors=['#f0f0f0', '#404040', '#808080'])
axes[1, 2].set_title('Win Rate (%)')
axes[1, 2].set_xlabel('Iteration')
axes[1, 2].set_ylabel('Percentage')
axes[1, 2].legend(loc='upper right')
axes[1, 2].set_ylim(0, 100)
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(MODEL_DIR / 'rl_training_curves.png', dpi=150)
plt.show()

print(f"학습 곡선 저장: {MODEL_DIR / 'rl_training_curves.png'}")